**Библиотеки**

In [16]:
import pandas as pd
import numpy as np
from utils import fetch_ohlcv_df, moex_fetch_ohlcv_df
import ccxt

exchange = ccxt.binance({"enableRateLimit": True})

**Матрица доходностей**

In [36]:
ticker_list = ['BTC/USDT','ETH/USDT','TON/USDT','BNB/USDT','XRP/USDT','KAVA/USDT']

In [37]:
df_pct_change = dict()
for ticker in ticker_list:
    df = fetch_ohlcv_df(exchange,ticker,'1M',"2025-01-01","2025-12-31")
    df_pct_change[f'{ticker}'] = df['close'].pct_change().dropna()

In [24]:
df = fetch_ohlcv_df(
    exchange,
    "TON/USDT",
    "1M",
    "2025-01-01",
    "2025-12-31"
)

In [40]:
df_pct_change = pd.DataFrame(df_pct_change)
df_pct_change

,BTC/USDT,ETH/USDT,TON/USDT,BNB/USDT,XRP/USDT,KAVA/USDT
timestamp,,,,,,
2025-02-01 00:00:00+00:00,-0.176508,-0.322146,-0.310545,-0.131962,-0.293403,-0.035738
2025-03-01 00:00:00+00:00,-0.021339,-0.185539,0.234675,0.028466,-0.025824,-0.002059
2025-04-01 00:00:00+00:00,0.140787,-0.015814,-0.230713,-0.008151,0.048280,-0.005044
2025-05-01 00:00:00+00:00,0.110647,0.409481,-0.004113,0.096903,-0.007760,-0.036636
2025-06-01 00:00:00+00:00,0.024425,-0.016847,-0.074968,-0.001687,0.028705,-0.043052
2025-07-01 00:00:00+00:00,0.080428,0.488004,0.211882,0.192175,0.351579,-0.049488
2025-08-01 00:00:00+00:00,-0.064940,0.187498,-0.103145,0.095143,-0.081624,-0.012096
2025-09-01 00:00:00+00:00,0.053605,-0.056168,-0.143760,0.175899,0.025615,-0.153846
2025-10-01 00:00:00+00:00,-0.038939,-0.071689,-0.157196,0.079712,-0.118414,-0.601132


**Ковариационная матрица**

In [ ]:
cov = df_pct_change.cov(ddof = 1)
cov

,BTC/USDT,ETH/USDT,TON/USDT,BNB/USDT,XRP/USDT,KAVA/USDT
BTC/USDT,0.011022,0.017130,0.008164,0.008546,0.012453,0.001631
ETH/USDT,0.017130,0.063442,0.024452,0.021143,0.029675,0.005379
TON/USDT,0.008164,0.024452,0.034054,0.012542,0.017898,0.000982
BNB/USDT,0.008546,0.021143,0.012542,0.014042,0.013158,-0.003584
XRP/USDT,0.012453,0.029675,0.017898,0.013158,0.026421,0.007618
KAVA/USDT,0.001631,0.005379,0.000982,-0.003584,0.007618,0.038023


**Риск и доходность(средняя по месяцу)**

In [42]:
std = df_pct_change.std(ddof = 1)
std

BTC/USDT     0.104987
ETH/USDT     0.251878
TON/USDT     0.184538
BNB/USDT     0.118497
XRP/USDT     0.162545
KAVA/USDT    0.194995
dtype: float64

In [43]:
returns = df_pct_change.mean()
returns

BTC/USDT    -0.008859
ETH/USDT     0.017053
TON/USDT    -0.076272
BNB/USDT     0.028888
XRP/USDT    -0.032662
KAVA/USDT   -0.123184
dtype: float64